In [2]:
%pip install beautifulsoup4 lxml


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 KB 299.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 51.7 kB/s eta 0:00:0000:0100:03m
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import csv

def crawl_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        html_content = response.text
        return html_content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_unique_hrefs(html_content):
    soup = BeautifulSoup(html_content, 'lxml')
    
    box_product_divs = soup.find_all('div', class_='box_product')
    
    hrefs = set()
    
    for div in box_product_divs:
        a_tag = div.find('a', href=True)
        if a_tag:
            href = a_tag['href']
            hrefs.add(href)
    
    return list(hrefs)

if __name__ == "__main__":
    list_product = []
    for page in range(0, 172):
        url = f"https://buudien.vn/home/Search/index.html?keyword=OCOP&page={page}"
        html_data = crawl_html(url)
        if html_data:
            unique_hrefs = extract_unique_hrefs(html_data)
            list_product.extend(unique_hrefs)
        else:
            print("Failed to retrieve the webpage.")
    list_product = list(set(list_product))
    with open('products.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Product URL'])
        for product in list_product:
            csvwriter.writerow([product])

    print("Product URLs have been written to products.csv")

Product URLs have been written to products.csv


In [5]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.8 kB/s eta 0:00:0000:0100:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 51.2 kB/s eta 0:00:0000:0100:09
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 KB 44.6 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import html 
import requests
from bs4 import BeautifulSoup
import csv

# Read the CSV file
products_df = pd.read_csv('products.csv')
    
def crawl_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Kiểm tra lỗi HTTP
        html_content = response.text
        return html_content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None
    
def extract_product_info(html_content):
    soup = BeautifulSoup(html_content, 'lxml')
    

    product_info = {}

    # get product name
    product_divs = soup.find_all('div', class_='name_product_details')
    for div in product_divs:
        product_info = {}
        
        h2_tag = div.find('h2')
        if h2_tag:
            product_info["product_name"] = h2_tag.get_text(strip=True)
    #get price        
    price_tag = soup.find('div', class_='price_cart').find('h3')
    if price_tag:
        product_info["price"] = price_tag.get_text(strip=True) 

    # get location
    location_tag = soup.find('div', class_='drop_kv_giaohang')
    if location_tag:
        product_info["location"] = location_tag.get_text(strip=True)
    
    #get store infor
    h3_tag = soup.find('div', class_='info_store').find('h3')

    #get description
    description = soup.find('div', class_='wp_content_tab_description_product')
    if description:
        product_info["description"] = " ".join(description.stripped_strings)
    
    if h3_tag:
        text = h3_tag.get_text(strip=True)
        decoded_text = html.unescape(text)
        product_info["store"] = decoded_text
    #get product image 
    img_tags = soup.find('div', class_='swiper-wrapper').find_all('img', src=True, attrs={'data-src': True})
    if img_tags:
        image_urls = [img['data-src'].split('?')[0] for img in img_tags]
        product_info["image_urls"] = image_urls
    return product_info

list_product_info = []
for product_url in products_df['Product URL']:
    url = f"https://buudien.vn{product_url}"
    html_data = crawl_html(url)
    if html_data:
        product_info = extract_product_info(html_data)
        list_product_info.append(product_info)
    else:
        print("Failed to retrieve the webpage.")
    
    
    # Write product information to CSV file
with open('product_info_new.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['product_name', 'price', 'location', 'store', 'description', "image_urls" ]
    csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
    csvwriter.writeheader()
    for product_info in list_product_info:
        csvwriter.writerow(product_info)

# print("Product information has been written to product_info.csv")

NameError: name 'csv' is not defined

In [6]:
import csv
with open('product_info_new.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['product_name', 'price', 'location', 'store', 'description', "image_urls" ]
    csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
    csvwriter.writeheader()
    for product_info in list_product_info:
        csvwriter.writerow(product_info)

In [4]:
list_product_info

[{'product_name': 'OCOP - Mắm Nhĩ Thượng Hạng Duy Trinh Quảng Nam 300ml',
  'price': '40.000đ',
  'location': 'Quảng Nam',
  'description': 'Mắm nhỉ thượng hạng được chế biến từ cá cơm than lựa chọn rât tỷ mỹ. Cá cơm phải to con dài 7-9cm, tươi ngon được rộn với muối Ninh Thuận theo tỷ lệ 3 cá : 1 muối và ủ trong thùng gỗ 12-18 tháng. Nước mắm thượng hạng có vị mặn đàu lưỡi và vị ngọt nơi cuốn họng tạo nên hương vị đặc trưng của nước mắm truyền thống. Được chế biến từ 100% cá, muối và đóng chai thủy tinh giúp hương vị nước mắm được bảo quản tốt hơn. Nước mắm thượng hạng là món quà biếu đầy ý nghĩa và sang trọng cho người thân!',
  'store': 'Nước Mắm Truyền Thống Duy Trinh',
  'image_urls': ['https://dyh48pub5c8mm.cloudfront.net/home/store/goods/1590/20231130/alioss_1590_2023113013553536464.png',
   'https://dyh48pub5c8mm.cloudfront.net/home/store/goods/1590/20231130/alioss_1590_2023113013555555794.pn

In [7]:
import requests
from bs4 import BeautifulSoup
import csv

def download_html_file(url, filename):
    html_content = crawl_html(url)
    if html_content:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(html_content)
        print(f"HTML content has been written to {filename}")
    else:
        print("Failed to retrieve the webpage.")


In [14]:

if __name__ == "__main__":
    list_product = []

    url = f"https://buudien.vn/home/Goods/index.html?goods_id=26024"
    html_data = download_html_file(url, "aaa1.html")



HTML content has been written to aaa1.html


In [30]:
import pandas as pd
import html 
# Read the CSV file
products_infor_df = pd.read_csv('product_info.csv')
    

In [31]:
products_infor_df

,product_name,price,location,store,description,image_urls
0,OCOP - Mắm Nhĩ Thượng Hạng Duy Trinh Quảng...,40.000đ,Quảng Nam,Nước Mắm Truyền Thống Duy Trinh,Mắm nhỉ thượng hạng được chế biến từ c...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
1,OCOP Bình Định Dầu đậu phộng Vân Nam 6 lít ( t...,550.000đ,Bình Định,BĐT Bình Định,100% nguyên chấtkhông chất bảo quảnCÔNG DỤNG1....,['https://dyh48pub5c8mm.cloudfront.net/home/st...
2,OCOP Lâm Đồng - Hạt mắc ca Di Linh - 500g dạng...,260.000đ,Lâm Đồng,HTX Liên kết Mắc ca Di Linh,Hạt mắc ca Di Linh sản xuất theo quy Trình Vie...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
3,OCOP - Nhang Thảo Dược Sạch ( Bồ Công Anh Hộp ...,110.000đ,Đồng Tháp,Công ty TNHH phát triển P & K,Sản phẩm nhang thảo dược đựơc chuyển giao công...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
4,OCOP - MIẾN DONG TÂN VIỆT Á,42.000đ,Cao Bằng,Nông sản Cao Bằng,MIẾN DONG PHIA ĐÉN CAO BẰNG - NGUYÊN CHẤT - HÀ...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
...,...,...,...,...,...,...
2047,Mít sấy Thịnh Phát Đạt (Túi 100gr),100.000đ,Hồ Chí Minh,Trái cây sấy Thịnh Phát Đạt,Sản phẩm được sản xuất từ thiết bị tiên tiến h...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
2048,"Bún khô Thanh Mai - SP OCOP Văn Yên, Yên Bái",14.000đ,NaN,BC Ga,"Bún khôlà món ăn rất tiệndụng, chúng ta có thể...",['https://dyh48pub5c8mm.cloudfront.net/home/st...
2049,Sữa tươi thanh trùng 480ml,26.000đ,Bà Rịa Vũng Tàu,CÔNG TY TNHH THỰC PHẨM LONG PHƯỚC,: Sữa tươi thanh trùng BRMILK được xử lý theo ...,['https://dyh48pub5c8mm.cloudfront.net/home/st...
2050,OCOP Bến Tre - Trang sức dừa Cocohand - Vòng t...,85.000đ,Mỏ Cày,Sản Phẩm OCOP Bến Tre,"CÂU CHUYỆN SẢN PHẨMGiai đoạn ươm mầmXin chào, ...",['https://dyh48pub5c8mm.cloudfront.net/home/st...
